In [1]:
import pandas as pd

In [2]:
df= pd.read_csv('data-NSE_FO_35000.csv')

In [3]:
df.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,OI
0,1,2024-06-28 03:45:00,24349.90,24403.85,24349.9,24403.85,450,50
1,2,2024-06-28 03:46:00,24399.05,24406.75,24389.7,24400.00,1250,50
2,3,2024-06-28 03:47:00,24400.00,24410.25,24392.0,24400.00,775,50
3,4,2024-06-28 03:48:00,24400.60,24405.00,24400.0,24405.00,275,2050
4,5,2024-06-28 03:49:00,24407.50,24424.90,24400.6,24424.90,2425,2050


In [4]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [5]:
# Convert the 'Date' column to datetime format (optional but recommended)
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
# Localize to UTC, then convert to +5:30
df['Date'] = df['Date'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')

In [7]:
# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

In [8]:
df.head()

,Open,High,Low,Close,Volume,OI
Date,,,,,,
2024-06-28 09:15:00+05:30,24349.90,24403.85,24349.9,24403.85,450,50
2024-06-28 09:16:00+05:30,24399.05,24406.75,24389.7,24400.00,1250,50
2024-06-28 09:17:00+05:30,24400.00,24410.25,24392.0,24400.00,775,50
2024-06-28 09:18:00+05:30,24400.60,24405.00,24400.0,24405.00,275,2050
2024-06-28 09:19:00+05:30,24407.50,24424.90,24400.6,24424.90,2425,2050


In [9]:
def closing(df):
  return df.Close

In [10]:
from backtesting import Strategy
from backtesting.lib import crossover


class Alphabots(Strategy):
    delta = 0.0005
    base_price = df['Close'][0]
    volume = 0.5
    def init(self):
      # self.base_price = self.data.Close[0]
      self.short_trade = 0
      self.long_trade = 0
      self.close_prices = self.I(closing, self.data)
      self.square_off = self.data.Close[374]

    def next(self):
      # if (self.close_prices == self.square_off) or (self.long_trade+self.short_trade==2):
      #   self.position.close()
      print(self.data)
      if self.close_prices > self.base_price*(1+self.delta):
        if self.short_trade < 1:
          if self.position.is_long or not self.position:
            self.position.close()
          self.sell(size = self.volume)
          self.short_trade += 1
      if self.close_prices < self.base_price*(1-self.delta):
        if self.long_trade < 1:
          if self.position.is_short or not self.position:
            self.position.close()  
          self.buy(size = self.volume)
          self.long_trade += 1



C:\Users\user\AppData\Local\Temp\ipykernel_9412\2250169048.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  base_price = df['Close'][0]


In [11]:
from backtesting import Backtest

bt = Backtest(df[:375], Alphabots, cash=100_000, commission=0.00)
stats = bt.run()


<Data i=2 (2024-06-28 09:17:00+05:30) Open=24400.0, High=24410.25, Low=24392.0, Close=24400.0, Volume=775.0, OI=50.0>
<Data i=3 (2024-06-28 09:18:00+05:30) Open=24400.6, High=24405.0, Low=24400.0, Close=24405.0, Volume=275.0, OI=2050.0>
<Data i=4 (2024-06-28 09:19:00+05:30) Open=24407.5, High=24424.9, Low=24400.6, Close=24424.9, Volume=2425.0, OI=2050.0>
<Data i=5 (2024-06-28 09:20:00+05:30) Open=24421.85, High=24421.85, Low=24408.0, Close=24411.3, Volume=1050.0, OI=2050.0>
<Data i=6 (2024-06-28 09:21:00+05:30) Open=24406.05, High=24406.05, Low=24400.0, Close=24405.0, Volume=475.0, OI=5250.0>
<Data i=7 (2024-06-28 09:22:00+05:30) Open=24405.05, High=24416.65, Low=24400.05, Close=24413.4, Volume=1150.0, OI=5250.0>
<Data i=8 (2024-06-28 09:23:00+05:30) Open=24415.0, High=24415.05, Low=24412.75, Close=24412.75, Volume=900.0, OI=5250.0>
<Data i=9 (2024-06-28 09:24:00+05:30) Open=24414.9, High=24414.9, Low=24413.0, Close=24413.0, Volume=275.0, OI=6850.0>
<Data i=10 (2024-06-28 09:25:00+05:3

In [161]:
stats

Start                     2024-06-28 09:15...
End                       2024-06-28 15:29...
Duration                      0 days 06:14:00
Exposure Time [%]                        48.8
Equity Final [$]                     100054.1
Equity Peak [$]                      100079.9
Return [%]                             0.0541
Buy & Hold Return [%]               -0.165753
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.119448
Avg. Drawdown [%]                    -0.04222
Max. Drawdown Duration        0 days 03:09:00
Avg. Drawdown Duration        0 days 01:14:00
# Trades                                    2
Win Rate [%]                             50.0
Best Trade [%]                       0.163583
Worst Trade [%]                     -0.052908
Avg. Trade [%]                    

In [162]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2,5,186,24421.85,24381.9,79.9,0.001636,2024-06-28 09:20:00+05:30,2024-06-28 12:21:00+05:30,0 days 03:01:00
1,2,186,187,24381.90,24369.0,-25.8,-0.000529,2024-06-28 12:21:00+05:30,2024-06-28 12:22:00+05:30,0 days 00:01:00


In [163]:
bt.plot()

d:\user\Documents\workspace\Alphabots\.venv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\user\Documents\workspace\Alphabots\.venv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\user\Documents\workspace\Alphabots\.venv\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
d:\user\Documents\workspace\Alphabots\.venv\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
d:\user\Documents\workspace\Alpha

GridPlot(id='p5870', ...)

In [123]:
stats, heatmap = bt.optimize(
                    delta=[i/1000 for i in range(1, 101)],
                    volume = [i/10 for i in range(1,6)],
                    maximize='Return [%]', max_tries=300,
                        random_state=0,
                        return_heatmap=True)

In [124]:
stats

Start                     2024-06-28 09:15...
End                       2024-06-28 15:29...
Duration                      0 days 06:14:00
Exposure Time [%]                   43.466667
Equity Final [$]                      99939.0
Equity Peak [$]                     100029.75
Return [%]                             -0.061
Buy & Hold Return [%]               -0.165753
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.091823
Avg. Drawdown [%]                   -0.051535
Max. Drawdown Duration        0 days 05:36:00
Avg. Drawdown Duration        0 days 02:54:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -0.249693
Worst Trade [%]                     -0.249693
Avg. Trade [%]                    

In [125]:
stats["_strategy"]

<Strategy Alphabots(delta=0.001,volume=0.4)>

In [89]:
bt.plot()

d:\user\Documents\workspace\Alphabots\.venv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\user\Documents\workspace\Alphabots\.venv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\user\Documents\workspace\Alphabots\.venv\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
d:\user\Documents\workspace\Alphabots\.venv\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
d:\user\Documents\workspace\Alpha

GridPlot(id='p2020', ...)